# Driver Aggression Neural Network (DANN)

Driver Aggression Neural Network is assigning an aggression value to a sorted set of sensory data. Driving is simulated in BeamNG v0.27 using their BeamNGpy open-source library.

In [ ]:
import pandas as pd
import torch

parquet_file_path = '../imu_data_2023_05_05_18_56_11.parquet'
data = pd.read_parquet(parquet_file_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Modify training data

- Group together data recorded from the same sensor
- Take around 100-1000 recorded data without the aggression values
- Make aggression values the label of the dataset
- Create a lot of training data by chunking the sorted (by timestamp) records.

In [ ]:
import numpy as np

# Assuming 'data' is your Scheme
# data = pd.DataFrame(columns=[
#     'imuId',
#     'vehicleAggression',
#     'time',
#     'pos',
#     'dirX',
#     'dirY',
#     'dirZ',
#     'angVel',
#     'angAccel',
#     'mass',
#     'accRaw',
#     'accSmooth'
# ])

# Function to split the data into chunks
def split_into_chunks(data, chunk_size):
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

# Group the data by 'imuId' and sort within each group by 'time'
grouped_data = data.groupby('imuId').apply(lambda x: x.sort_values('time')).reset_index(drop=True)

# Set the desired chunk size (number of records per chunk)
chunk_size = 200

# Split the data into chunks and assign the 'vehicleAggression' value as the label
training_data = []
for imu_id, group in grouped_data.groupby('imuId'):
    group.dropna()
    chunks = split_into_chunks(group, chunk_size)
    for chunk in chunks:
        if len(chunk) >= chunk_size:
            label = chunk['vehicleAggression'].iloc[0]
            first_timestamp = chunk['time'].iloc[0]
            adjusted_time = chunk['time'] - first_timestamp
            
            # Separate list columns into individual columns
            # pos_df = pd.DataFrame(chunk['pos'].tolist(), columns=['posX', 'posY', 'posZ'], index=chunk.index)
            dir_x_df = pd.DataFrame(chunk['dirX'].tolist(), columns=['dirXX', 'dirXY', 'dirXZ'], index=chunk.index)
            dir_y_df = pd.DataFrame(chunk['dirY'].tolist(), columns=['dirYX', 'dirYY', 'dirYZ'], index=chunk.index)
            dir_z_df = pd.DataFrame(chunk['dirZ'].tolist(), columns=['dirZX', 'dirZY', 'dirZZ'], index=chunk.index)
            acc_raw_df = pd.DataFrame(chunk['accRaw'].tolist(), columns=['accRawX', 'accRawY', 'accRawZ'], index=chunk.index)
            acc_smooth_df = pd.DataFrame(chunk['accSmooth'].tolist(), columns=['accSmoothX', 'accSmoothY', 'accSmoothZ'], index=chunk.index)
            ang_vel_df = pd.DataFrame(chunk['angVel'].tolist(), columns=['angVelX', 'angVelY', 'angVelZ'], index=chunk.index)
            ang_accel_df = pd.DataFrame(chunk['angAccel'].tolist(), columns=['angAccelX', 'angAccelY', 'angAccelZ'], index=chunk.index)
            
            expanded_chunk = pd.concat(
                [
                    chunk,
                    # pos_df,
                    dir_x_df,
                    dir_y_df,
                    dir_z_df,
                    acc_raw_df,
                    acc_smooth_df,
                    ang_vel_df,
                    ang_accel_df
                ],
                axis=1
            )
            
            updated_chunk = (
                expanded_chunk.assign(time=adjusted_time)
                .drop(['imuId', 'mass', 'vehicleAggression', 'pos', 'dirX', 'dirY', 'dirZ', 'angVel', 'angAccel', 'accRaw', 'accSmooth'], axis=1)
            )
            
            training_data.append({'data': updated_chunk, 'label': label})

# Convert the list of dictionaries to a DataFrame
training_data_df = pd.DataFrame(training_data)

# Example of a single training set
# Set display options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", 100)  # Set the maximum column width to 100 characters
pd.set_option("display.expand_frame_repr", False)
print(training_data_df.loc[0, 'data'])
pd.reset_option("all")


## Translating the training data to learn

In [ ]:
from sklearn.model_selection import train_test_split

# Get the data and labels from the training_data_df
X = np.stack(training_data_df['data'].apply(lambda x: x.to_numpy()).to_numpy())
y = training_data_df['label'].to_numpy()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

## Learn with PyTorch

- Create a TensorDataset
- Create a DataLoader, which shuffles the data
- Create a simple neural net (torch.nn.Sequential) which uses CUDA while training
- Train the neural net with the data provided
- Evaluate the net with the test data

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

X_train_2d = X_train.reshape(X_train.shape[0], -1)
X_test_2d = X_test.reshape(X_test.shape[0], -1)
X_train_rnn = X_train.reshape(-1, X_train.shape[1], X_train.shape[2])
X_test_rnn = X_test.reshape(-1, X_test.shape[1], X_test.shape[2])
print("X_train_2d shape:", X_train_2d.shape)
print("X_test_2d shape:", X_test_2d.shape)

# Create tensors from the padded data
X_train_tensor = torch.tensor(X_train).permute(0, 2, 1).to(device)
X_train_tensor_2d = torch.tensor(X_train_2d).permute(0, 1).to(device)
y_train_tensor = torch.tensor(y_train).to(device)
X_test_tensor = torch.tensor(X_test).permute(0, 2, 1).to(device)
X_test_tensor_2d = torch.tensor(X_test_2d).permute(0, 1).to(device)
y_test_tensor = torch.tensor(y_test).to(device)

# Create a DataLoader for the training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataset_2d = TensorDataset(X_train_tensor_2d, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_dataset_2d = TensorDataset(X_test_tensor_2d, y_test_tensor)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
train_loader_2d = DataLoader(train_dataset_2d, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
test_loader_2d = DataLoader(test_dataset_2d, batch_size=batch_size)

# rnn_batch_size = 1 # Use for RNN
# X_train_tensor_rnn = torch.tensor(X_train_rnn).permute(0, 2, 1).to(device)
# X_test_tensor_rnn = torch.tensor(X_test_rnn).permute(0, 2, 1).to(device)
# rnn_train_dataset = TensorDataset(X_train_rnn, y_train_tensor)
# rnn_test_dataset = TensorDataset(X_test_rnn, y_test_tensor)
# rnn_train_loader = DataLoader(rnn_train_dataset, batch_size=rnn_batch_size, shuffle=True)
# rnn_test_loader = DataLoader(rnn_test_dataset, batch_size=rnn_batch_size)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# # Linear Regression
# lr = LinearRegression()
# lr.fit(X_train_2d, y_train)
# y_pred_lr = lr.predict(X_test_2d)

# # Random Forest
# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(X_train_2d, y_train)
# y_pred_rf = rf.predict(X_test_2d)

# # Support Vector Regression
# svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)
# svr.fit(X_train_2d, y_train)
# y_pred_svr = svr.predict(X_test_2d)

# # Evaluation
# mse_lr = mean_squared_error(y_test, y_pred_lr)
# r2_lr = r2_score(y_test, y_pred_lr)

# mse_rf = mean_squared_error(y_test, y_pred_rf)
# r2_rf = r2_score(y_test, y_pred_rf)

# mse_svr = mean_squared_error(y_test, y_pred_svr)
# r2_svr = r2_score(y_test, y_pred_svr)

# print("Linear Regression: MSE = {:.4f}, R2 = {:.4f}".format(mse_lr, r2_lr))
# print("Random Forest: MSE = {:.4f}, R2 = {:.4f}".format(mse_rf, r2_rf))
# print("Support Vector Regression: MSE = {:.4f}, R2 = {:.4f}".format(mse_svr, r2_svr))


In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Prepare the dataset
train_data = lgb.Dataset(X_train_2d, label=y_train)
test_data = lgb.Dataset(X_test_2d, label=y_test, reference=train_data)

# Set up the model parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'device_type': 'gpu',
    'n_jobs': -1,
}

# Train the model
gbm = lgb.train(params, train_data, valid_sets=test_data, num_boost_round=1000, early_stopping_rounds=50)

# Make predictions
y_pred_lgb = gbm.predict(X_test_2d, num_iteration=gbm.best_iteration)

# Calculate metrics
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
r2_lgb = r2_score(y_test, y_pred_lgb)
mae_lgb = mean_absolute_error(y_test, y_pred_lgb)

print("LightGBM Regression: MSE = {:.4f}, R2 = {:.4f}, MAE = {:.4f}".format(mse_lgb, r2_lgb, mae_lgb))

In [ ]:
import torch.nn as nn

# Define the linear regression model
class LinearRegressor(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressor, self).__init__()
        self.linear = nn.Linear(input_size, output_size).to(device)

    def forward(self, x):
        return self.linear(x)

In [ ]:
import torch.optim as optim

input_size = X_train_2d.shape[1]
output_size = 1
model = LinearRegressor(input_size, output_size).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, labels in train_loader_2d:
        inputs, labels = inputs.float().to(device), labels.float().to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

model.eval()
predictions, actuals = [], []

with torch.no_grad():
    for inputs, targets in test_loader_2d:
        inputs, targets = inputs.float().to(device), targets.float().to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(targets.cpu().numpy())

y_pred_torch = np.array(predictions).flatten()
y_pred_torch[np.isnan(y_pred_torch)] = 0
y_test_torch = np.array(actuals).flatten()
mse_torch = mean_squared_error(y_test_torch, y_pred_torch)
r2_torch = r2_score(y_test_torch, y_pred_torch)

print("PyTorch Linear Regression: MSE = {:.4f}, R2 = {:.4f}".format(mse_torch, r2_torch))

In [ ]:
import torch.nn as nn

# Define the CNN architecture
class CNNRegressor(nn.Module):
    def __init__(self, input_channels, num_filters, kernel_size, pool_size, hidden_units, dropout_rate, device):
        super(CNNRegressor, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, num_filters, kernel_size),
            nn.BatchNorm1d(num_filters),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.conv2 = nn.Sequential(
            nn.Conv1d(num_filters, num_filters * 2, kernel_size),
            nn.BatchNorm1d(num_filters * 2),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.conv3 = nn.Sequential(
            nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size),
            nn.BatchNorm1d(num_filters * 4),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.flatten = nn.Flatten()

        conv1_out_size = (chunk_size - kernel_size + 1) // pool_size
        conv2_out_size = (conv1_out_size - kernel_size + 1) // pool_size
        conv3_out_size = (conv2_out_size - kernel_size + 1) // pool_size

        self.fc1 = nn.Sequential(
            nn.Linear(num_filters * 4 * conv3_out_size, hidden_units),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        ).to(device)

        self.fc2 = nn.Sequential(
            nn.Linear(hidden_units, hidden_units // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        ).to(device)

        self.fc3 = nn.Linear(hidden_units // 2, 1).to(device)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [ ]:
import optuna
import random
import torch.optim as optim
import torch.nn.functional as F

def train_and_eval_cnn(trial):
    # Suggest hyperparameters using the trial object
    # num_filters = trial.suggest_int("num_filters", 256, 512)
    # kernel_size = trial.suggest_int("kernel_size", 3, 5)
    # pool_size = trial.suggest_int("pool_size", 2, 4)
    # hidden_units = trial.suggest_int("hidden_units", 24, 92)
    # dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    # learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    
    # Value: 0.028036039788275957
    # Params: {'num_filters': 437, 'kernel_size': 4, 'pool_size': 4, 'hidden_units': 73, 'dropout_rate': 0.1657660256105821, 'learning_rate': 3.964884325777312e-05}
    num_filters = 437
    kernel_size = 4
    pool_size = 4
    hidden_units = 73
    dropout_rate = 0.1657660256105821
    learning_rate = 3.964884325777312e-05
    
    # Create the model
    input_channels = X_train.shape[2]
    net = CNNRegressor(input_channels, num_filters, kernel_size, pool_size, hidden_units, dropout_rate, device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    num_epochs = 10
    
    net.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.float().to(device), labels.float().to(device)
    
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
        # print(f"Epoch {epoch+1}/{num_epochs} Loss: {running_loss/len(train_loader)}")

    net.eval()
    predictions, actuals = [], []
    
    with torch.no_grad():
        test_loss = 0.0
        for inputs, labels in test_loader:
            inputs, labels = inputs.float().to(device), labels.float().to(device)
            outputs = net(inputs)

            loss = criterion(outputs.squeeze(), labels)
            test_loss += loss.item()
            
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

        print(f"Test Loss: {test_loss/len(test_loader)}")

        # Calculate the average absolute difference
        labels_tensor = torch.tensor(predictions, device=device)
        # actuals_tensor = torch.tensor(actuals, device=device)
        # average_difference = F.l1_loss(labels_tensor, actuals_tensor)
        # Calculate the average absolute fake difference using MAE
        fake_labels_tensor = torch.tensor([random.uniform(0.2, 1.0) for _ in predictions], device=device)
        average_fake_difference = F.l1_loss(fake_labels_tensor, labels_tensor)
        average_difference = mean_absolute_error(actuals, predictions)
        print(f"Average Absolute Difference: {average_difference.item()}")
        print(f"Average Absolute Fake Difference: {average_fake_difference}")
    
    return test_loss/len(test_loader)

train_and_eval_cnn(None)

In [ ]:
class RNNRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, sequence_length):
        super(RNNRegressor, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.sequence_length = sequence_length
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, output_size)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        
        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [ ]:
def train_and_eval_rnn(trial):
    
    # Hyperparameters
    input_size = X_train.shape[2]  # Change input size to 22
    output_size = 1
    sequence_length = X_train.shape[1]  # Add sequence length (200)
    
    # num_layers = trial.suggest_int("num_layers", 256, 512)
    # hidden_size = trial.suggest_int("hidden_size", 24, 92)
    # learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    
    num_layers = 256
    hidden_size = 24
    learning_rate = 1e-5
    
    # Model, Loss, and Optimizer
    model = model = RNNRegressor(input_size, hidden_size, num_layers, output_size, sequence_length).to(device)  # Pass sequence_length
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    num_epochs = 10
    print(f"Number of epochs: {num_epochs}")
    model.train()
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(rnn_train_loader):
            inputs, labels = inputs.float().to(device), labels.float().to(device)
    
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            print(f"{i/len(rnn_train_loader)} Test Loss: {loss.item()}")

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
        # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    print("Finished training")
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        test_loss = 0.0
        for i, (inputs, labels) in enumerate(rnn_test_loader):
            inputs, labels = inputs.float().to(device), labels.float().to(device)
            outputs = model(inputs)
            
            loss = criterion(outputs.squeeze(), labels)
            test_loss += loss.item()
            print(f"{i}. Test Loss: {test_loss/len(rnn_test_loader)}")
            
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

        print(f"Test Loss: {test_loss/len(test_loader)}")

        # Calculate the average absolute difference
        labels_tensor = torch.tensor(predictions, device=device)
        average_difference = F.l1_loss(labels_tensor, labels)
        # Calculate the average absolute fake difference using MAE
        fake_labels_tensor = torch.tensor([random.uniform(0.2, 1.0) for _ in predictions], device=device)
        average_fake_difference = F.l1_loss(fake_labels_tensor, labels_tensor)
        print(f"Average Absolute Difference: {average_difference.item()}")
        print(f"Average Absolute Fake Difference: {average_fake_difference}")
    
    return test_loss/len(test_loader)

# train_and_eval_rnn(None)

## Hyperparameter optimization via Optuna

In [ ]:
# study = optuna.create_study(direction="minimize")
# study.optimize(train_and_eval_rnn, n_trials=20)  # You can adjust the number of trials depending on your computational resources

In [ ]:
# Get the best 5 trials
# completed_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])
# best_trials = sorted(completed_trials, key=lambda t: t.value)[:5]

# # Print the best 5 trials' parameters and their respective values
# for i, trial in enumerate(best_trials):
#     print(f"Best trial {i + 1}:")
#     print(f"  Value: {trial.value}")
#     print(f"  Params: {trial.params}")